In [15]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, MaxPooling1D, Flatten, Dense


In [33]:
data = pd.read_csv("C:/Users/Aayush Garg/Downloads/output whole data indexed (1).csv")

In [35]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

data = pd.read_csv("C:/Users/Aayush Garg/Downloads/output whole data indexed (1).csv")  


data = data.dropna(subset=['clause_text', 'clause_type'])


label_encoder = LabelEncoder()
data['label'] = label_encoder.fit_transform(data['clause_type'])

# Step 5: Split the Data into Training and Testing Sets
X = data['clause_text']
y = data['label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 6: Tokenization and Padding
max_sequence_length = 1000  
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)
X_train = tokenizer.texts_to_sequences(X_train)
X_train = pad_sequences(X_train, maxlen=max_sequence_length)
X_test = tokenizer.texts_to_sequences(X_test)
X_test = pad_sequences(X_test, maxlen=max_sequence_length)

vocab_size = len(tokenizer.word_index) + 1




In [36]:
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(
    X,  # tokenized and padded text data
    y,  #  encoded labels
    test_size=0.2,  
    random_state=42 
)



In [38]:
max_index = max(tokenizer.word_index.values())
print(f"Maximum index value in the tokenizer's word index: {max_index}")


Maximum index value in the tokenizer's word index: 23020


In [39]:
embedding_layer = Embedding(
    input_dim=vocab_size,  
    output_dim=100,  
    input_length=max_sequence_length
)


In [40]:
vocab_size = len(tokenizer.word_index) + 1  # Add 1 for the padding token


In [41]:
vocab_size

23021

In [43]:
vocab_size = len(tokenizer.word_index) + 1  


In [44]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Sequential
from keras.layers import Embedding, Conv1D, MaxPooling1D, Flatten, Dense
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences


data = pd.read_csv("filtered_data.csv1")  


# Tokenization and Padding
max_sequence_length = 1000
tokenizer = Tokenizer()
tokenizer.fit_on_texts(data['clause_text'])
X = tokenizer.texts_to_sequences(data['clause_text'])
X = pad_sequences(X, maxlen=max_sequence_length)

# Label Encoding
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(data['clause_type'])

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Build the CNN model
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=100, input_length=max_sequence_length)) 
model.add(Conv1D(filters=128, kernel_size=5, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(len(label_encoder.classes_), activation='softmax'))  

# Compile the model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=5, batch_size=32)

# Evaluate the model on the test data
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Loss: {loss:.4f}, Test Accuracy: {accuracy:.4f}')


Epoch 1/5
747/747 [==============================] - 147s 196ms/step - loss: 1.9956 - accuracy: 0.4765 - val_loss: 1.0166 - val_accuracy: 0.7422
Epoch 2/5
747/747 [==============================] - 148s 199ms/step - loss: 0.6825 - accuracy: 0.8191 - val_loss: 0.7596 - val_accuracy: 0.8164
Epoch 3/5
747/747 [==============================] - 146s 195ms/step - loss: 0.3545 - accuracy: 0.8979 - val_loss: 0.7796 - val_accuracy: 0.8052
Epoch 4/5
747/747 [==============================] - 142s 190ms/step - loss: 0.2288 - accuracy: 0.9324 - val_loss: 0.9131 - val_accuracy: 0.8042
Epoch 5/5
187/187 [==============================] - 6s 30ms/step - loss: 0.9344 - accuracy: 0.8127
Test Loss: 0.9344, Test Accuracy: 0.8127


In [45]:

model.save('your_model.h5')


C:\Users\Aayush Garg\anaconda3\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [47]:
from keras.models import load_model

model = load_model("your_model.h5")


In [57]:
new_text = input("Enter The Text ---") 
new_text_sequences = tokenizer.texts_to_sequences(new_text)
new_text_padded = pad_sequences(new_text_sequences, maxlen=max_sequence_length)


Enter The Text ---During the Access Term, Gene Logic, at its expense, will provide Genaissance: (a) any subsequent release or version of the Gene Logic Software (including bug fixes, patches and maintenance releases) included in the GeneExpress(TM) Product as Gene Logic makes such releases available to its other customers; and (b) support and maintenance of Gene Logic Software included in the GeneExpress(TM) Product through reasonable consultation via telephone, fax, electronic mail or otherwise during Gene Logic's normal business hours (8:00 a.m. to 5:00 p.m. Eastern Time on regular U.S. business days, holidays excepted) on specific problems that arise in the delivery and use of the GeneExpress(TM) Product. Should Genaissance request on-site support or maintenance it will pay Gene Logic's reasonable out-of-pocket expenses and a daily consultation fee of ************************ during the Access Term.


In [58]:

predictions = model.predict(new_text_padded)

predicted_class_index = np.argmax(predictions[0])


predicted_clause_type = label_encoder.classes_[predicted_class_index]

print(f'Predicted Clause Type: {predicted_clause_type}')


29/29 [==============================] - 1s 38ms/step
Predicted Clause Type: Definitions
